# Duplicate detection with LSH

***

This code performs duplicate detection in the 'restaurant.data.csv' dataset using Locality-Sensitive Hashing (LSH). The steps involved in this process are as follows:

Preprocessing: The text data in the 'name', 'addr', 'city', 'phone', and 'type' columns is combined into a single 'Combined' column. The combined text is then preprocessed by converting to lowercase, removing numbers and punctuations, removing stopwords, and stripping whitespace.

Shingling: The preprocessed text is then shingled into sets of k-grams (in this case, k=5) using the 'get_shingles()' function.

Find more about Shingling :<a href = "https://towardsai.net/p/l/text-similarity-using-k-shingling-minhashing-and-lshlocality-sensitive-hashing">Here</a>

Signature creation: A MinHash signature is created for each shingled text using the 'MinHash()' function from the datasketch library. The signature is created with 128 permutations.

Indexing: The signatures are indexed using LSH with a similarity threshold of 0.6 using the 'MinHashLSH()' function from the datasketch library.

Querying: For each document, the shingled text is hashed using MinHash to create a signature, and the LSH index is queried to find similar documents. Documents with similar MinHash signatures are considered potential duplicates.

Output: The code outputs a list of potential duplicate pairs of documents along with their corresponding rows in the original 'restaurant.data.csv' dataset.

### Required Libraries

This code imports several libraries that will be used in the project:

* pandas: a library for data manipulation and analysis
* datasketch: a library for probabilistic data structures such as MinHash and LSH
* re: a module for regular expression operations
* string: a module containing a collection of string constants and functions for string manipulation.

In [1]:
import pandas as pd
import datasketch as ds
import re
import string

### Reading the dataset

This code reads a CSV file named 'restaurant.data.csv' into a Pandas DataFrame called 'df'. The CSV file should be in the same directory as this notebook.

In [2]:
df = pd.read_csv('restaurant.data.csv')

In [3]:
df.head()

,name,addr,city,phone,type,class
0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,american,0
1,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,steakhouses,0
2,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,american,1
3,art's deli,12224 ventura blvd.,studio city,818-762-1221,delis,1
4,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,californian,2


### Drop any rows with missing data

This code removes all rows from the DataFrame df which contain at least one missing value (i.e. NaN). The inplace=True parameter ensures that the original DataFrame is modified instead of creating a new copy.

In [4]:
df.dropna(inplace=True)

In [5]:
df.reset_index(inplace = True, drop = True)

### Taking the copy of our dataset for later use. 

This line of code creates a copy of the original dataframe df and assigns it to a new variable old_df. The copy() method is used to create a new copy of the dataframe so that any changes made to old_df will not affect the original df.

In [6]:
old_df = df.copy()

### Combine columns into one text column

This code creates a new column in the dataframe called "Combined" by concatenating values from multiple columns of the dataframe: "name", "addr", "city", "phone", and "type". The resulting column will contain a combination of these values for each row of the dataframe, which will be used to identify similar documents.

In [7]:
df['Combined'] = df['name'] + df['addr']+ df['city'] + df['phone'] + df['type']

### Function to preprocess the text

This code defines a function named preprocess that takes a string text as input and applies several text preprocessing techniques to it:

Converts the text to lowercase using the lower() method.
Removes all digits from the text using the re.sub() method and regular expressions.
Removes all punctuation from the text using the translate() method and the string.punctuation constant.
Removes leading and trailing whitespace from the text using the strip() method.
Removes stopwords (commonly used words that do not add much meaning to a text) from the text using a list of predefined stopwords and a list comprehension.
The processed text is then returned from the function.

In [8]:
def preprocess(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'\d', '', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove whitespace
    text = text.strip()
    
    # Remove stopwords
    stopwords = ['the', 'and', 'in', 'of', 'on', 'at', 'a', 'an']
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    return text

### Apply preprocessing to the Combined column

This line of code applies the preprocess() function to each value in the 'Combined' column of the df DataFrame using the apply() method. The preprocess() function performs several text preprocessing steps to clean the text data, such as converting the text to lowercase, removing numbers and punctuation, removing whitespace, and removing stop words. The resulting cleaned text is then stored back in the 'Combined' column of the DataFrame.

In [9]:
df['Combined'] = df['Combined'].apply(preprocess)

### Define a function for shingling text

Shingling is a technique used in text analysis to transform text into a set of overlapping subsequences of characters, called shingles. In this project, shingling is used to break down the restaurant information into a set of shingles of length k.

Shingling is useful in this project because it enables the use of MinHash and Locality Sensitive Hashing (LSH) algorithms, which are used to efficiently compare large sets of data. By breaking down the text into shingles, we can represent each document as a set of shingles and use the Jaccard similarity coefficient to compare the similarity between two sets of shingles. This allows us to efficiently compare the similarity between restaurant records based on their name, address, city, phone, and type, even if the records have different lengths or wordings.

This is a function named get_shingles that takes two arguments, a text string and an integer k, which is set to 5 by default. The function returns a set of shingles, which are substrings of length k created from the input text string.

The function first initializes an empty set to store the shingles. It then converts the input text string to lowercase. It iterates over the characters in the string using a loop, with the loop variable i ranging from 0 to the length of the string minus k plus 1. In each iteration, the function creates a substring of length k starting at position i and adds it to the shingles set. Finally, the function returns the shingles set.

In [10]:
def get_shingles(text, k=5):
    """Splits text into shingles of length k."""
    # Initialize an empty set to store shingles
    shingles = set()
    
    # Convert the text to lowercase
    text = text.lower()
    
    # Iterate over the text to create shingles of length k
    for i in range(len(text) - k + 1):
        shingle = text[i:i + k]
        shingles.add(shingle)
    return shingles

### Creating MinHash signatures using shingles for each document in the dataset

MinHash is a probabilistic algorithm used for estimating the similarity between two sets. It uses hash functions to create a signature for each set, which is a binary vector of fixed length. The signature is created by iterating over all elements in the set, and for each element, applying the hash functions to obtain a hash value. The minimum hash value is selected for each hash function, and the resulting vector of minimum hash values is used as the signature.

In this project, MinHash signatures are used to represent the shingles extracted from the combined text of each row in the input dataframe. Each row is first preprocessed, including lowercasing, removing numbers, punctuation, whitespace, and stopwords. Then, shingles of length k=5 are extracted using the get_shingles function. A MinHash object with 128 hash functions is created for each shingle set, and the object is updated with each shingle in the set. The resulting signature is a vector of 128 minimum hash values.

The significance of using MinHash signatures in this project is that they provide an efficient way to estimate the similarity between two sets. Instead of comparing each shingle in one set with every shingle in another set, which can be computationally expensive for large sets, the Jaccard similarity between the two sets can be approximated using the MinHash signatures. Locality-Sensitive Hashing (LSH) is then used to group similar signatures together, allowing for efficient identification of potential duplicates in the input dataframe.

This code creates MinHash signatures for each document in the dataset. It starts by defining the number of permutations to be used in the MinHash algorithm, which affects its accuracy. The Jaccard similarity threshold is also defined, which is the minimum similarity between two documents required to be considered as duplicates.

Then, for each row in the dataframe, the get_shingles function is called to generate shingles for that row's text. A MinHash object is created with the number of permutations defined earlier, and the object is updated with each shingle in the shingle set. Finally, the MinHash signature for that document is appended to the signatures list.

In [11]:
# Extract shingles and create MinHash signatures for each document
num_perm = 228      # num_perm is the number of permutations used in the MinHash algorithm, which affects the accuracy of the algorithm. It is set to 128.
threshold = 0.6     # threshold is the Jaccard similarity threshold for identifying similar documents. It is set to 0.6.
signatures = []     # An empty list called signatures is initialized to hold the MinHash signatures for each document.
for i, row in df.iterrows():
    # get_shingles function is called on the 'Combined' column of the current row to generate shingles for that row.
    shingles = get_shingles(row['Combined'])
    
    # A MinHash object is created with the num_perm specified earlier.
    minhash = ds.MinHash(num_perm=num_perm)
    
    for shingle in shingles:
        minhash.update(shingle.encode('utf8'))        # The MinHash object is updated with each shingle in the shingle set.
        
    # The final MinHash signature is appended to the signatures list.
    signatures.append(minhash)

### Initialize LSH index

This line of code creates an instance of the MinHashLSH class from the datasketch library. MinHashLSH is an implementation of Locality-Sensitive Hashing (LSH) algorithm that is used for efficient near-neighbor search in high-dimensional spaces.

The threshold parameter sets the Jaccard similarity threshold for identifying similar documents. The num_perm parameter sets the number of permutations used in the MinHash algorithm, which affects the accuracy of the algorithm.

By setting the threshold and num_perm parameters, we are defining the similarity metric and the level of accuracy of our LSH index. The MinHashLSH object will be used to index the MinHash signatures generated in the previous step so that we can efficiently search for near-duplicate documents in our dataset.

In [12]:
lsh = ds.lsh.MinHashLSH(threshold=threshold, num_perm=num_perm)

### Finding Duplicates using Locality-Sensitive Hashing (LSH) Algorithm

First, the signatures of all the documents are indexed using the LSH algorithm with the specified threshold and number of permutations.
Then, for each document in the dataset, its shingles are extracted and a MinHash signature is generated using the same number of permutations as before.
The LSH index is queried with this MinHash signature to find any similar documents within the threshold similarity.
Finally, any duplicate document pairs found are stored in the 'duplicates' list as tuples of their indices. The comparison only goes one way to avoid duplicate pairs (i.e., only (i, j) is added, not (j, i) if both are similar).

In [13]:
# Index the signatures
for i, signature in enumerate(signatures):
    lsh.insert(str(i), signature)

# Query the LSH index for similar documents
duplicates = []
for i, row in df.iterrows():
    shingles = get_shingles(row['Combined'])
    minhash = ds.MinHash(num_perm=num_perm)
    for shingle in shingles:
        minhash.update(shingle.encode('utf8'))
    query_results = lsh.query(minhash)
    for j in query_results:
        if i < int(j):
            duplicates.append((i, int(j)))

### Printing Duplicate Rows

This code prints the pairs of similar documents found using Locality-Sensitive Hashing (LSH) on the dataset. For each pair of similar documents, it prints the row numbers and the corresponding values of the columns 'name', 'addr', 'city', 'phone', and 'type' from the original dataset. The pairs of documents are printed with a separator line to visually distinguish them.

In [14]:
# Print the similar documents
for pair in duplicates:
    print("Rows: ", pair,"---",list(old_df.iloc[pair[0]]), '\t', list(old_df.iloc[pair[1]]))
    print('--------------------------------------------------------------------------')

Rows:  (0, 1) --- ["arnie morton's of chicago", '435 s. la cienega blv.', 'los angeles', '310/246-1501', 'american', 0] 	 ["arnie morton's of chicago", '435 s. la cienega blvd.', 'los angeles', '310-246-1501', 'steakhouses', 0]
--------------------------------------------------------------------------
Rows:  (4, 5) --- ['hotel bel-air', '701 stone canyon rd.', 'bel air', '310/472-1211', 'californian', 2] 	 ['bel-air hotel', '701 stone canyon rd.', 'bel air', '310-472-1211', 'californian', 2]
--------------------------------------------------------------------------
Rows:  (6, 7) --- ['cafe bizou', '14016 ventura blvd.', 'sherman oaks', '818/788-3536', 'french', 3] 	 ['cafe bizou', '14016 ventura blvd.', 'sherman oaks', '818-788-3536', 'french bistro', 3]
--------------------------------------------------------------------------
Rows:  (12, 13) --- ['citrus', '6703 melrose ave.', 'los angeles', '213/857-0034', 'californian', 6] 	 ['citrus', '6703 melrose ave.', 'los angeles', '213-857-0

***

### Evaluation

This code calculates the accuracy of the duplicate detection algorithm used in the project. It first initializes three variables: correct, incorrect, and total, which are set to 0.

Then, it iterates over each pair of duplicates found in the dataset using a for loop. For each pair, it compares the class label (stored in the 'class' column) of the first and second rows. If the class labels are the same, it increments the correct variable. If they are different, it increments the incorrect variable. The total variable is incremented for each pair of duplicates, regardless of whether they are classified correctly or not.

Finally, the code calculates the accuracy as correct divided by total, and prints it out as a formatted string. The accuracy is a measure of how many pairs of duplicates were correctly classified by the algorithm, compared to the total number of pairs.

In [15]:
correct = 0
incorrect = 0
total = 0

for first,second in duplicates:
    if df.loc[first,'class'] == df.loc[second,'class']:
        correct += 1
    else:
        incorrect += 1
    total += 1

accuracy = correct / total

print(f"Accuracy score is {accuracy}")

Accuracy score is 0.22666666666666666


***

This project uses a machine learning algorithm, specifically the MinHash algorithm, to identify similar documents. The MinHash algorithm is a probabilistic algorithm that is commonly used for similarity search in large datasets. Although it is not a traditional machine learning algorithm that involves training a model on labeled data, it is still considered part of the machine learning field because it uses mathematical techniques to perform a task that would be difficult or impossible to do manually.

In addition, preprocessing techniques such as lowercasing, removing punctuation, and removing stopwords are often used in natural language processing (NLP) tasks, which are a common area of application in machine learning.

To add to that, Minhash is often used in machine learning and data mining applications as a preprocessing step for similarity search and clustering tasks. It can be used to efficiently compute similarity between large datasets without having to compare every pair of objects, which can be computationally expensive. This makes it a useful tool for tasks such as duplicate detection, recommendation systems, and content-based search.

Additionally, you could add other machine learning algorithms to this project. One possible approach would be to use the MinHash signatures as features for a supervised learning model, such as a binary classification model to classify documents as either duplicates or non-duplicates. You could use the labeled duplicates found using LSH as training data for this model. Alternatively, you could use unsupervised learning techniques to cluster similar documents based on their MinHash signatures.

***

Suggestions to potentially improve this project:

* Use more advanced natural language processing techniques: The current preprocessing step is quite basic, and there are many more advanced techniques that can be used to further clean and normalize the text data. For example, stemming, lemmatization, and part-of-speech tagging could be used to extract more meaningful features from the text.

* Use a more advanced similarity measure: The Jaccard similarity measure used in this project is a simple and effective measure for comparing sets of shingles, but there are many more advanced similarity measures that could be used, such as cosine similarity or the Soft-TFIDF measure.

* Incorporate supervised learning: While Minhash and LSH are unsupervised techniques, supervised learning algorithms could be used to further improve the accuracy of the document similarity task. For example, a classifier could be trained to distinguish between similar and dissimilar documents based on features extracted from the text, and this classifier could be used to filter the candidate pairs identified by the Minhash and LSH algorithms.

* Use larger datasets: The dataset used in this project is relatively small, so the results may not be representative of how the algorithms perform on larger datasets. Using larger datasets could help to evaluate the scalability and accuracy of the algorithms.

* Tune hyperparameters: The current project uses default values for the hyperparameters of the Minhash and LSH algorithms. Tuning these hyperparameters could help to optimize the performance of the algorithms for the specific dataset and task. For example, the number of permutations used in the Minhash algorithm and the threshold used in the LSH algorithm could be tuned to maximize the accuracy of the document similarity task.

***

Here are a few additional suggestions for detecting similar entries in a dataset using machine learning algorithms:

Clustering: Clustering algorithms can be used to group similar entries together based on the features they have in common. One approach could be to use a clustering algorithm such as k-means or hierarchical clustering on a subset of the features in the dataset, and then manually inspect the resulting clusters to see if they contain any entries that appear to be duplicates.

Neural Networks: A neural network can be trained to classify pairs of entries as either duplicates or not, based on the features of the entries. This approach would involve training a binary classifier using a neural network architecture such as a multi-layer perceptron or a convolutional neural network.

Ensemble Learning: An ensemble learning approach could involve training multiple machine learning algorithms on different subsets of the features in the dataset, and then combining their results to make a final determination of whether two entries are duplicates or not. This approach can help to reduce the impact of noise in the data and increase the overall accuracy of the duplicate detection algorithm.

Graph-based Algorithms: A graph-based approach could involve representing the dataset as a graph, where each entry is a node and edges between nodes represent some measure of similarity between them. Graph-based algorithms such as the label propagation algorithm or the PageRank algorithm can be used to identify groups of entries that are highly connected and likely to be duplicates.

Siamese Networks: Siamese networks are neural networks that can learn to compare two inputs and output a similarity score based on how similar the inputs are. Siamese networks can be used to compare pairs of entries in the dataset and identify those that are likely to be duplicates. This approach involves training a neural network with pairs of entries, where each pair is labeled as either a duplicate or not. Once the network is trained, it can be used to compare new pairs of entries and predict whether they are duplicates or not.





***